<a href="https://colab.research.google.com/github/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/Beam_Selector_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Back End

### Library Import

In [ ]:
import pandas as pd

### Function Definitions


#### Deformation Limit Functions

#### Stress Limit Functions

In [ ]:
def findStress(M, ):
    print("Hello")


##### Stress Limit Check Function

In [ ]:
def stressLimitCheck(M, result):
    for libNum in range(0,14):
        beamLib = pd.read_excel('/content/beamLibrary.xlsx', libNum)
        if libNum == 8 or libNum == 13 or libNum == 14:
            for h in range(0,len(beamLib.index)-1):
                Stress = (M*(beamLib.iat[h,1]/2000))/((beamLib.iat[h,5]*10**-8)*10**6)
                if Stress < stressLimit:
                    result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    break

        elif libNum == 12:
            for h in range(0,len(beamLib.index)-1):
                Stress = (M*(beamLib.iat[h,1]/2000))/((beamLib.iat[h,4]*10**-8)*10**6)
                if Stress < stressLimit:
                    result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    break
    
        elif libNum == 7 or libNum == 9 or libNum == 10 or libNum == 11:
            for h in range(0,len(beamLib.index)-1):
                Stress = (M*(beamLib.iat[h,1]/2000))/((beamLib.iat[h,3]*10**-8)*10**6)
                if Stress < stressLimit:
                    result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    break
        
        elif libNum == 0 or libNum == 1 or libNum == 2 or libNum == 3 or libNum == 4 or libNum == 5 or libNum == 6:
            for h in range(0,len(beamLib.index)-1):
                Stress = (M*(beamLib.iat[h,1]/2000))/((beamLib.iat[h,6]*10**-8)*10**6)
                if Stress < stressLimit:
                    result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,0]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    break
    return result

##### Case 1: Stress Limit

In [ ]:
def case1_stress():
    M = force*lengthOfBeam/4 # Bending Moment
    result = ["You selected Case 1, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheck(M, result)
    return _result

##### Case 2: Stress Limit

In [ ]:
def case2_stress():
    M = force*lengthOfBeam # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheck(M, result)
    return _result

# **Input Form** 

In [ ]:
caseNumber = 2 #@param ["1", "2"] {type:"raw"}
checkFor = "Stress" #@param ["Stress", "Deformation"]
force =  50000#@param {type:"number"}
lengthOfBeam =  5#@param {type:"number"}
length_a = 0 #@param {type:"number"}
length_b = 0 #@param {type:"number"}
length_c = 0 #@param {type:"number"}
youngsModulus = 203 #@param {type:"number"}
stressLimit = 190 #@param {type:"number"}
deformationLimit = 0 #@param {type:"number"}


# Interface

In [ ]:
if (caseNumber == 2):
    if(checkFor == "Stress"):
        result = case2_stress()
        


# **RESULT**

In [ ]:
print(*result,sep='\n')

You selected Case 2, Stress Limit
The minimum required beam and the stress are: 
HEB 280 ...  Stress: 181.63 MPa.
HEA 320 ...  Stress: 168.99 MPa.
HEM 240 ...  Stress: 138.95 MPa.
IPE 450 ...  Stress: 166.72 MPa.
IPN 400 ...  Stress: 171.17 MPa.
Square Tube CF 1997 300 ...  Stress: 169.87 MPa.
Square Tube HF 1997 300 ...  Stress: 157.23 MPa.
Circular Tube 323.9 ...  Stress: 182.87 MPa.
